In [1]:
import time
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

In [20]:
quotes = [
        {'code': 'EURUSD', 'market': '5', 'em': '83'},
        {'code': 'GBPUSD', 'market': '5', 'em': '86'},
        {'code': 'USDCHF', 'market': '5', 'em': '85'},
        {'code': 'USDJPY', 'market': '5', 'em': '87'},
        {'code': 'USDSEK', 'market': '5', 'em': '398509'},
        {'code': 'AUDUSD', 'market': '5', 'em': '66699'},
        {'code': 'NZDUSD', 'market': '5', 'em': '181425'},
        {'code': 'USDCAD', 'market': '5', 'em': '66700'}
]

In [21]:
days = 400

edate = datetime.now()
delta = timedelta(days=days)
sdate = edate - delta

e1 = edate.strftime('%d.%m.%Y')
s1 = sdate.strftime('%d.%m.%Y')

e2 = edate.strftime('%y.%m.%d')
s2 = sdate.strftime('%y.%m.%d')

edate = datetime.timetuple(edate)
sdate = datetime.timetuple(sdate)
ey, em, ed = edate[:3]
sy, sm, sd = sdate[:3]

In [22]:
def get_url(code, s1, e1, sd, sm, sy, s2, ed, em, ey, e2, market, e):
    sd, sm, sy, ed, em, ey = str(sd), str(sm-1), str(sy), str(ed), str(em-1), str(ey) 
    return ('http://export.finam.ru/{0}_{1}_{2}.csv?market={11}&em={12}&code={0}&apply=0&df={3}&mf={4}&yf={5}&' +\
           'from={6}&dt={7}&mt={8}&yt={9}&to={10}&p=7&f={0}_{1}_{2}&e=.csv&cn={0}&dtf=1&tmf=1&MSOR=1&mstime=on&' +\
           'mstimever=1&sep=3&sep2=1&datf=5&at=1').format(code, s1, e1, sd, sm, sy, s2, ed, em, ey, e2, market, e)

In [23]:
def loading_to_csv(quotes):
    for quote in quotes:
        code, market, e = quote['code'], quote['market'], quote['em']
        url = get_url(code, s1, e1, sd, sm, sy, s2, ed, em, ey, e2, market, e)
        data = pd.read_csv(url, parse_dates={'Date & Time': [0, 1]}, index_col=0, sep=';')
        data.rename(lambda x: str.lower(x[1:-1]), axis='columns', inplace=True)
        data.to_csv(quote['code'] + '.csv')

In [24]:
def make_dataset(fname, short, long, k, d):
    data = pd.read_csv(fname, index_col=0)
    data['A'] = data.close.rolling(window=short).mean()
    data['B'] = data.close.rolling(window=long).mean()
    data['%K'] = (data.close - data.low.rolling(window=k).min())/ \
                 (data.high.rolling(window=k).max() - data.low.rolling(window=k).min()) * 100
    data['%D'] = data['%K'].rolling(window=d).mean()
    return data

In [25]:
def paint(data, quote):
    plt.figure(figsize=(14, 8))
    plt.plot(data.index, data.close, data.index, data.A, data.index, data.B)
    plt.xlabel('Date')
    plt.ylabel('Quote')
    plt.legend()
    plt.title(quote['code'])
    plt.grid()
    plt.show()

In [9]:
#loading_to_csv(quotes)

In [70]:
def run(q, short, long, k, d, back, amp):
    fname = q['code']  + '.csv'
    data = make_dataset(fname, short, long, k, d)
    deals = pd.DataFrame(columns=['dir', 'open', 'close', 'sl', 'tp'])

    for i in range(len(data.close)):
        _ = max([short, long, back])
        rng = range(0, _)
        if i in rng:
            continue
        if data.A[i-2] < data.B[i-2] and data.A[i-1] > data.B[i-1] and data['%K'][i-1] < 80:
            sl = data.low[i-back: i].min()
            tp = data.open[i] + (data.open[i] - sl) * amp
            index = deals.shape[0]
            deals.loc[index] = [1, data.open[i], 0, sl, tp]
        if data.A[i-2] > data.B[i-2] and data.A[i-1] < data.B[i-1] and data['%K'][i-1] > 20:
            sl = data.high[i-back: i].max()
            tp = data.open[i] - (sl - data.open[i]) * amp
            index = deals.shape[0]
            deals.loc[index] = [-1, data.open[i], 0, sl, tp]
        else:
            for d in range(deals.shape[0]):
                if deals.loc[d].close == 0:
                    if deals.loc[d].dir == 1 and data.high[i] >= deals.loc[d].tp:
                        deals.loc[d].close == deals.loc[d].tp
                    if deals.loc[d].dir == 1 and data.low[i] <= deals.loc[d].sl:
                        deals.loc[d].close == deals.loc[d].sl
                    if deals.loc[d].dir == -1 and data.low[i] <= deals.loc[d].tp:
                        deals.loc[d].close == deals.loc[d].tp
                    if deals.loc[d].dir == -1 and data.high[i] >= deals.loc[d].sl:
                        deals.loc[d].close == deals.loc[d].sl

    dls = deals.copy()
    dls.close[dls.close == 0] = dls.sl
    dls['dif'] = (dls['close'] - dls['open']) * dls['dir']
    profit = round(dls.dif.sum() * 100000, 2)
    
    output = '{}   Short SMA: {:<3} Long SMA: {:<3} Lookback: {:<3} Amplifier: {:<3} Profit: {:>15}\n'\
    .format(q['code'], short, long, back, amp, profit)
    
    print(output)
    
    with open('results.txt', 'a') as f:
        f.write(output)

In [71]:
short = 5
long = 20
k = 14
d = 3
back = 5
amp = 1

In [72]:
for q in quotes[:1]:
    run(q, short, long, k, d, back, amp)

KeyboardInterrupt: 